NCF Recommender with Explict Feedback

In [1]:
#Context-Aware Neural Collaborative Filtering (CA-NCF)
#Author: Isam Mashhour Al Jawarneh
#Date: 30 January 2019
from zoo.models.recommendation import *
from zoo.models.recommendation.utils import *
from zoo.common.nncontext import init_nncontext
import os
import sys
import datetime as dt
from bigdl.dataset.transformer import *
from bigdl.dataset.base import *
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *
from bigdl.optim import *
from bigdl.util.common import *
import matplotlib

matplotlib.use('agg')
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
sc = init_nncontext(create_spark_conf().setAppName("CA_NCF Movielens"))

In [3]:
from scipy import stats
import numpy as np
import pandas as pd
#import researchpy as rp

In [4]:
df = pd.read_csv("datasets/movies/movielens1.csv", names=['UserID', 'ItemID', 'rating','dayOfWeek', 'month','year'], header=None, usecols=[0,1,2,3,9,10])[['UserID', 'ItemID', 'rating','dayOfWeek','month' ,'year']]

In [5]:
# for constructing objects containing items, the contexts and context conditions

class MyClass(object):
    def __init__(self, item,condition,context):
        self.number = number
        self.item = item
        self.condition = condition
        self.context = context

In [6]:
#some_list = []
#some_list.append(MyClass('2','weekend','dayOfWeek'))
#some_list.append(MyClass('3', 'M', 'gender'))
#for obj in some_list:
 #   print(obj.item)
  #  print(obj.condition)
   # print(obj.context)

######ITEM SPLITTING Procedure############
## as described in :
## Experimental evaluation of context-dependent collaborative filtering using item splitting
## https://dl.acm.org/citation.cfm?id=2579997
#df = df1.head(1000)
## output a list (item, condition, context), where condition is the context condition that will be used for splitting the item

#profiling data
some_list = []
contextFactors = ['year','dayOfWeek']
dflist = df['ItemID'].unique().tolist()
#array_length = len(contextFactors)
#print(array_length)
#j = 1
cmax = 0.0
con = ''
context = ''
for i in dflist:
    cmax = 0.0
    for j in contextFactors:
        #print(j)
        conditionList = df[j].unique().tolist()
        #array_length1 = len(conditionList)
        #print(array_length1)
        for k in conditionList:
            #print(k)
            item1 = df[(df['ItemID'] == i) & (df[j] == k)]
            item1.reset_index(inplace= True)  
            item2 = df[(df['ItemID'] == i) & (df[j] != k)]
            item2.reset_index(inplace= True)
            l = stats.ttest_ind(item1['rating'], item2['rating'])
            #if l.statistic > 4.0:
            #print(i), print(l)
            if (l.statistic > cmax) :
                cmax = l.statistic
                con = k
                context = j
            #print(cmax)
        
    #print('finished')
    if cmax > 4.0:
        print('item ' + str(i) + ' cmax is ' + str(cmax) + ' con ' + str(con) + ' context ' + str(context))
        some_list.append(MyClass(str(i),str(con),str(context)))

      
  



In [7]:
#this acts as a pre-trained model
# this is a partial list only of the items to split. known by profiling the data
# for a full list of items to split: disable this cell and enable the previous cell
some_list = []
some_list.append(MyClass('3578','2000','year'))
some_list.append(MyClass('3793','2000','year'))
some_list.append(MyClass('3948','2000','year'))
some_list.append(MyClass('3751','2000','year'))
some_list.append(MyClass('106','weekday','dayOfWeek'))
some_list.append(MyClass('815','weekend','dayOfWeek'))


In [8]:
import os
#should be the same path in Jupyter where this notebook exists
notebook_path = os.path.abspath("movielens.ipynb")

In [9]:
sqlContext = SQLContext(sc)
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import col, udf

#CHANGE rating to Rating for 1million dataset
Rating = Row("UserID", "ItemID", "rating","dayOfWeek","gender","age","occupation","dayOfMonth","dayOfYear","month","year")
#UserID,ItemID,Rating,UserState,UserTimeZone,ItemCity,ItemState,ItemTimeZone,TripType

ratingsget = os.path.join(os.path.dirname(notebook_path), "datasets/movies/movielens1.csv")
ratings= sc.textFile(ratingsget) \
    .map(lambda line: line.split(",")[0:11])\
    .map(lambda line: (line[0], line[1], int(line[2]), line[3],line[4],line[5], line[6],line[7],line[8],line[9],line[10]))\
    .map(lambda r: Rating(*r))
ratingDF_OLD = sqlContext.createDataFrame(ratings)
ratingDF_OLD.show(5)

someDFtt = ratingDF_OLD.select(col("UserID")).distinct()

#max_movie_id_alt = someDFt.count()

#print("UserID :   " + str(someDFtt.count()))

someDFtt1 = ratingDF_OLD.select(col("ItemID")).distinct()

#max_movie_id_alt = someDFt.count()

#print("ItemID :   " + str(someDFtt1.count()))


+------+------+------+---------+------+---+----------+----------+---------+-----+----+
|UserID|ItemID|rating|dayOfWeek|gender|age|occupation|dayOfMonth|dayOfYear|month|year|
+------+------+------+---------+------+---+----------+----------+---------+-----+----+
|     1|  1193|     5|  weekend|     F|  1|        10|        31|      366|   11|2000|
|     1|   661|     3|  weekend|     F|  1|        10|        31|      366|   11|2000|
|     1|   914|     3|  weekend|     F|  1|        10|        31|      366|   11|2000|
|     1|  3408|     4|  weekend|     F|  1|        10|        31|      366|   11|2000|
|     1|  2355|     5|  weekend|     F|  1|        10|         7|        7|    0|2001|
+------+------+------+---------+------+---+----------+----------+---------+-----+----+
only showing top 5 rows



In [10]:
#context = 'dayOfWeek'
populateList = []
#if context == 'year':
def condition(i):
    switcher={
            '2000':["2000", "2001","2002","2003"],
            '2001':["2001", "2000","2002","2003"],
            '2002':["2002", "2001","2000","2003"],
            '2003':["2003", "2001","2002","2000"],
            'weekend':["weekend", "weekday"],
            'weekday':["weekday", "weekend"],
            'M':["M","F"],
            'F':["F","M"],
         }
    return switcher.get(i,"Invalid year")


In [11]:
from pyspark.sql.functions import col, udf
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Binarizer
from pyspark.sql.functions import rand 

indexer = StringIndexer(inputCol="UserID", outputCol="newUserID1")
indexer1 = StringIndexer(inputCol="ItemID", outputCol="indexedItemID2")
indexer2 = StringIndexer(inputCol="artItemID1", outputCol="artItemID2")
binarizer22 = Binarizer(threshold=1.0, inputCol="locationIndexed", outputCol="binarized_location")

In [12]:
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, udf
df_list = []
toDiscard_df_list = []

for obj in some_list:
    itemid= obj.item
    ctxCond = obj.condition
    ctx = obj.context
    partialDF = ratingDF_OLD.withColumn("newUserID1",col("UserID")).withColumn("indexedItemID2",col("ItemID")).select("*").where((col("ItemID")==itemid) )
    toDiscard_df_list.append(partialDF)

    populateList = condition(ctxCond)
    print(populateList)
    print(ctx)
    general_udf = udf(lambda somename: categorical_from_vocab_list(somename, populateList, start=1))
    partialDF_indexed = partialDF.withColumn("locationIndexed", general_udf(col(ctx)).cast("double"))

    pipeline1 = Pipeline(stages = [binarizer22])
    allDF_temp =  pipeline1.fit(partialDF_indexed).transform(partialDF_indexed)
    allDF_temp.select('ItemID','binarized_location','gender','dayOfWeek','year','indexedItemID2').show(5)

    bucket_cross_udf = udf(lambda feature1, feature2: hash_bucket(str(feature1) + "_" + str(feature2), bucket_size=10000))
    df_list.append(allDF_temp.withColumn("artItemID1", bucket_cross_udf(col("indexedItemID2"), col("binarized_location")).cast("int") + 1))

['2000', '2001', '2002', '2003']
year
+------+------------------+------+---------+----+--------------+
|ItemID|binarized_location|gender|dayOfWeek|year|indexedItemID2|
+------+------------------+------+---------+----+--------------+
|  3578|               0.0|     M|  weekend|2000|          3578|
|  3578|               0.0|     M|  weekend|2000|          3578|
|  3578|               0.0|     F|  weekend|2000|          3578|
|  3578|               0.0|     M|  weekend|2000|          3578|
|  3578|               0.0|     M|  weekend|2000|          3578|
+------+------------------+------+---------+----+--------------+
only showing top 5 rows

['2000', '2001', '2002', '2003']
year
+------+------------------+------+---------+----+--------------+
|ItemID|binarized_location|gender|dayOfWeek|year|indexedItemID2|
+------+------------------+------+---------+----+--------------+
|  3793|               0.0|     M|  weekend|2000|          3793|
|  3793|               0.0|     M|  weekend|2000|     

In [13]:
allDF_prepare = reduce(DataFrame.unionAll, df_list).cache()

In [14]:
allDF_prepare.show(1)

+------+------+------+---------+------+---+----------+----------+---------+-----+----+----------+--------------+---------------+------------------+----------+
|UserID|ItemID|rating|dayOfWeek|gender|age|occupation|dayOfMonth|dayOfYear|month|year|newUserID1|indexedItemID2|locationIndexed|binarized_location|artItemID1|
+------+------+------+---------+------+---+----------+----------+---------+-----+----+----------+--------------+---------------+------------------+----------+
|     2|  3578|     5|  weekend|     M| 56|        16|        31|      366|   11|2000|         2|          3578|            1.0|               0.0|      1049|
+------+------+------+---------+------+---+----------+----------+---------+-----+----+----------+--------------+---------------+------------------+----------+
only showing top 1 row



In [15]:
from pyspark.sql.functions import lit
toDiscard_df_all = reduce(DataFrame.unionAll, toDiscard_df_list).cache()
#print(toDiscard_df_all.select(col('artItemID1')).distinct().count())
toDiscard_df_all_sub = toDiscard_df_all.select("*")
ratingDF_OLD_sub = ratingDF_OLD.withColumn("newUserID1",col("UserID")).withColumn("indexedItemID2",col("ItemID"))
remainingDF = ratingDF_OLD_sub.subtract(toDiscard_df_all_sub)

remainingDF_indexed = remainingDF.withColumn('binarized_location',lit(1.0))
bucket_cross_udf1 = udf(lambda feature1, feature2: hash_bucket(str(feature1) + "_" + str(feature2), bucket_size=1200000))
remainingDF_indexed_all = remainingDF_indexed.withColumn("artItemID1", bucket_cross_udf1(col("ItemID"), col("binarized_location")).cast("int") + 1)

In [16]:
remainingDF_indexed_all.show(1)

+------+------+------+---------+------+---+----------+----------+---------+-----+----+----------+--------------+------------------+----------+
|UserID|ItemID|rating|dayOfWeek|gender|age|occupation|dayOfMonth|dayOfYear|month|year|newUserID1|indexedItemID2|binarized_location|artItemID1|
+------+------+------+---------+------+---+----------+----------+---------+-----+----+----------+--------------+------------------+----------+
|     1|  1836|     5|  weekend|     F|  1|        10|        31|      366|   11|2000|         1|          1836|               1.0|   1146344|
+------+------+------+---------+------+---+----------+----------+---------+-----+----+----------+--------------+------------------+----------+
only showing top 1 row



In [17]:

remainingDF_indexed_all_sub = remainingDF_indexed_all.select(col('newUserID1') ,col('indexedItemID2'),col("UserID"),
                                        col('Rating'),col('ItemID'),col('artItemID1'),col("binarized_location"))
allDF_prepare_sub_sub = allDF_prepare.select(col("newUserID1") ,col("indexedItemID2"),col("UserID"),
                                        col("Rating"),col("ItemID"),col("artItemID1"),col("binarized_location"))
allDF_difinite = remainingDF_indexed_all_sub.union(allDF_prepare_sub_sub).cache()

In [18]:
someDF_art = allDF_difinite.select(col("artItemID1")).distinct()
print(someDF_art.count())

3706


In [19]:
pipeline2 = Pipeline(stages = [indexer2])

allDF =  pipeline2.fit(allDF_difinite).transform(allDF_difinite)

someDF1 = allDF.select((col("newUserID1").cast("int") ).alias("newUserID"),col("indexedItemID2").cast("int"),
                       col("Rating"),col("ItemID"),(col("artItemID2").cast("int") + 1).alias("aritid"))
someDF2 = allDF.select((col("newUserID1").cast("int")).alias("newUserID"),(col("artItemID2").cast("int") + 1).alias("artItemID"),col("Rating"))#.distinct()

someDF_OLD = someDF2.orderBy(rand())#.select("*").limit(5000000)


In [21]:
# CHANGE discard items with less than 20 ratings (interactions)
import pyspark.sql.functions as f
from pyspark.sql import Window
w = Window.partitionBy('artItemID')
someDF_NEW = someDF_OLD.withColumn('count_artItemID', f.count('newUserID').over(w))\
    .where(f.col('count_artItemID') > 100)\
    .drop('count_artItemID')#.select("*").limit(5000000)


In [22]:
w = Window.partitionBy('newUserID')
someDF = someDF_NEW.withColumn('count_newUserID', f.count('artItemID').over(w))\
    .where(f.col('count_newUserID') > 20)\
    .drop('count_newUserID')


In [23]:
someDFt3 = someDF.select(col("newUserID")).distinct()

max_user_id1 =  someDF.agg({"newUserID": "max"}).collect()[0][0]

someDFt4 = someDF.select(col("artItemID")).distinct()

max_item_id1 = someDF.agg({"artItemID": "max"}).collect()[0][0]

print(max_user_id1, max_item_id1)

6040 2010


In [27]:
def build_sample(user_id, item_id, rating):
    sample = Sample.from_ndarray(np.array([user_id, item_id]), np.array([rating]))
    return UserItemFeature(user_id, item_id, sample)
#CA-NCF settings
pairFeatureRdds_CANCF = someDF.rdd.map(lambda x: build_sample(x[0], x[1],x[2]))
#pairFeatureRdds = someDF.rdd.map(lambda x: build_sample(x[0], x[1],x[2]))

#CHANGE repartition1
pairFeatureRdds_CANCF.repartition(1)
pairFeatureRdds_CANCF.take(3)

In [28]:
#see more parameters here:
#https://github.com/intel-analytics/analytics-zoo/blob/master/docs/docs/APIGuide/Models/recommendation.md
#CHANGE ADD user_embed=20, item_embed=20, 
ncf = NeuralCF(user_count=max_user_id1, item_count=max_item_id1, class_num=5, hidden_layers=[20,10],include_mf = False)#, include_mf = True,mf_embed = 10)  user_embed=20, item_embed=20, 

creating: createZooNeuralCF


In [29]:
import time
millis = int(round(time.time() * 1000))
#CA-NCF settings
trainPairFeatureRdds_CANCF, valPairFeatureRdds_CANCF = pairFeatureRdds_CANCF.randomSplit([0.8, 0.2], seed = millis)
valPairFeatureRdds_CANCF.cache()
train_rdd_CANCF= trainPairFeatureRdds_CANCF.map(lambda pair_feature: pair_feature.sample)
val_rdd_CANCF= valPairFeatureRdds_CANCF.map(lambda pair_feature: pair_feature.sample)


In [30]:
##predicting user-item pairs using CA-NCF
predictUserItem_CANCF = ncf.predict_user_item_pair(valPairFeatureRdds_CANCF)
for ans in predictUserItem_CANCF.take(5): print(ans)

UserItemPrediction [user_id: 148, item_id: 1084, prediction: 4, probability: 0.27006351024788056]
UserItemPrediction [user_id: 148, item_id: 853, prediction: 4, probability: 0.27096964846008004]
UserItemPrediction [user_id: 148, item_id: 756, prediction: 4, probability: 0.2715697420252746]
UserItemPrediction [user_id: 148, item_id: 606, prediction: 4, probability: 0.2712912773787427]
UserItemPrediction [user_id: 148, item_id: 1210, prediction: 4, probability: 0.2708641380489278]


In [31]:
#Recommending 5 items for every user
recItemsForUsers_CANCF = ncf.recommend_for_user(valPairFeatureRdds_CANCF, 5)
for ans in recItemsForUsers_CANCF.take(10): print(ans)

UserItemPrediction [user_id: 1200, item_id: 82, prediction: 4, probability: 0.2727277485333273]
UserItemPrediction [user_id: 1200, item_id: 23, prediction: 4, probability: 0.27139912183679626]
UserItemPrediction [user_id: 1200, item_id: 14, prediction: 4, probability: 0.2707353980416912]
UserItemPrediction [user_id: 1200, item_id: 106, prediction: 4, probability: 0.2707231664058888]
UserItemPrediction [user_id: 1200, item_id: 84, prediction: 4, probability: 0.2707103544406774]
UserItemPrediction [user_id: 2200, item_id: 432, prediction: 4, probability: 0.2723625294827841]
UserItemPrediction [user_id: 2200, item_id: 22, prediction: 4, probability: 0.27105952780326453]
UserItemPrediction [user_id: 2200, item_id: 630, prediction: 4, probability: 0.2709920024347923]
UserItemPrediction [user_id: 2200, item_id: 100, prediction: 4, probability: 0.2708530952576944]
UserItemPrediction [user_id: 2200, item_id: 579, prediction: 4, probability: 0.2705317923256243]
